**Peliculas/Cine**

Topicos:
- Recursos
- Condiciónes de evnetos
- Eventos compartidos

**Enunciado**:

Modelar una sala de cine  con un mostrador para la venta de entradas a tres tipos de peliculas diferentes, tener en cuenta que se modela solo para un espectáculo.
- Las personas llegan en momentos aleatorios e intentan comprar un numero aleatorio de boletos (1-6) para una pelicula al azar.

- Cuando se agota las entradas a una pelicula, las personas que esperan por un boleto salen de la cola.

- La sala de cine es un Contenedor que abarca todos los procesos.
- El mostrador es un Recurso con la capacidad de uno.
- Finalmente, si la ultima persona desea comprar más boletos de los existentes se le vende solo los que tiene el mostrador.


In [31]:
import collections
import random
import simpy

VELOCIDAD_LLEGADA = 42
NUM_BOLETO = 50
TIEMPO_SIMULACION = 120
QUEUE={}

def ventaBoletos(env, num_boletos, pelicula, teatro, id):
  with teatro.contador.request() as turno:
    resultado = yield turno | teatro.sold_out[pelicula]
    if turno not in resultado:
      teatro.num_renegados[pelicula] += 1
      QUEUE[id]=[0,env.now, '']
      return      

    if teatro.num_boletos[pelicula] < num_boletos:
      yield env.timeout(0.5) 
      QUEUE[id]=[0,env.now, '']
      return
    else:
      QUEUE[id]=[1,env.now, pelicula]

    teatro.num_boletos[pelicula] -= num_boletos
    if teatro.num_boletos[pelicula] < 2:
      teatro.sold_out[pelicula].succeed()
      teatro.tiempo_agotado[pelicula] = env.now
      teatro.num_boletos[pelicula] = 0
    
    yield env.timeout(1)


def llegadaClientes(env, teatro):
  id=0
  while True:
    yield env.timeout(random.expovariate(1/0.5))
    pelicula = random.choices(teatro.peliculas, teatro.probabilidad, k=1)
    num_boletos = random.randint(1, 6)
    if teatro.num_boletos[pelicula[0]]:
      env.process(ventaBoletos(env, num_boletos, pelicula[0], teatro, id))
      id+=1


Teatro = collections.namedtuple('Teatro', 'contador, peliculas, probabilidad, num_boletos, sold_out, tiempo_agotado, num_renegados')

print('Teatro Carlos Crespi - UPS')
env = simpy.Environment()

contador = simpy.Resource(env,capacity=1)
peliculas = ['Conjuro 3', 'Rapidos y Furiosos 10', 'Pulp Fictions']
probabilidad=[0.1, 0.3, 0.6]
num_boletos = {pelicula: NUM_BOLETO for pelicula in peliculas}
sold_out = {pelicula: env.event() for pelicula in peliculas}
tiempo_agotado = {pelicula: None for pelicula in peliculas}
num_renegados = {pelicula: 0 for pelicula in peliculas}

teatro = Teatro(contador, peliculas, probabilidad, num_boletos, sold_out, tiempo_agotado, num_renegados)
env.process(llegadaClientes(env, teatro))
env.run(until=TIEMPO_SIMULACION)

# Analisis y resultados
for pelicula in peliculas:
  if teatro.sold_out[pelicula]:
    print('Pelicula: %s se agoto en el tiempo %.1f despues de salir a la venta' %(pelicula, teatro.tiempo_agotado[pelicula]))
    print('Numero de personas que salieron de la fila/renegados %s' %teatro.num_renegados[pelicula])


Teatro Carlos Crespi - UPS
Pelicula: Conjuro 3 se agoto en el tiempo 95.8 despues de salir a la venta
Numero de personas que salieron de la fila/renegados 0
Pelicula: Rapidos y Furiosos 10 se agoto en el tiempo 34.0 despues de salir a la venta
Numero de personas que salieron de la fila/renegados 0
Pelicula: Pulp Fictions se agoto en el tiempo 19.0 despues de salir a la venta
Numero de personas que salieron de la fila/renegados 4


**Tarea/Practica**

Al modelo de simulación de la venta de boletos de un teatro/cine agregar la simulación grafica 2D/3D utilizando cualquier libreria grafica, por ejemplo pygame/tkinter, etc. generar un informe en formato PDF que me permita evidenciar la simulación y generar una grafica/reporte de los resultados.

In [37]:
import simpy
import tkinter as tk
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg

class ClockAndData: 
    def __init__(self, canvas, x1, y1, x2, y2, time): 
        self.x1 = x1
        self.y1 = y1
        self.x2 = x2
        self.y2 = y2
        self.canvas = canvas
        data_plot.draw()
        self.canvas.update() 

    def tick(self, time): 
        a1.cla()
        
        self.y1 = self.y1+1
        self.y2 = self.y2+2

        bars_x = [self.x1, self.x2]
        bars_y = [self.y1, self.y2]

        a3.cla()
        a3.set_xlabel("Time")
        a3.set_ylabel("Arrivals")
        a3.bar(bars_x, bars_y)

        data_plot.draw()
        self.canvas.update() 
        # Re-draw the the clock and data fields on the canvas. Also update the Matplotlib charts. 

main = tk.Tk()
main.title("Simulacion Cine")
main.config(bg="#fff")

top_frame = tk.Frame(main)
top_frame.pack(side=tk.TOP, expand = False)

canvas = tk.Canvas(main, width = 1300, height = 350, bg = "white")
canvas.pack(side=tk.BOTTOM, expand = False)

f = plt.Figure(figsize=(1, 2), dpi=72)
a3 = f.add_subplot(121)
a3.plot()
a1 = f.add_subplot(222)
a1.plot()

data_plot = FigureCanvasTkAgg(f, master=main)
data_plot.get_tk_widget().config(height = 400)
data_plot.get_tk_widget().pack(side=tk.BOTTOM, fill=tk.BOTH, expand=True)

canvas.pack()

clock = ClockAndData(canvas, 10, 320, 20, 400, 0) 

def create_clock(env):
    while True: 
        yield env.timeout(1) 
        clock.tick(env.now) 

env = simpy.Environment()
env.process(create_clock(env)) 
env.run(until = 30)

main.mainloop()

TclError: invalid command name "pyimage22"